In [1]:
import ipywidgets as widgets
from IPython.core.display import display, HTML
import pandas as pd

# 스타일 및 레이아웃 설정
style = {'description_width': 'initial'}
layout_input = widgets.Layout(width='250px', margin='5px 10px')
layout_button = widgets.Layout(width='220px', height='40px', margin='20px 0')
layout_output = widgets.Layout(padding='10px', margin='10px 0', background='white', height='auto')
layout_main_box = widgets.Layout(border='1px solid #007BFF', padding='20px', align_items='center', width='750px')  # 가로폭 조절

# 입력 위젯 생성
전세금_위젯 = widgets.FloatText(description='전세금 (만원):', style=style, layout=layout_input)
연이자율_위젯 = widgets.FloatText(description='연 이자율 (%):', style=style, layout=layout_input)
월세_보증금_위젯 = widgets.FloatText(description='월세 보증금 (만원):', style=style, layout=layout_input)
월세금_위젯 = widgets.FloatText(description='월세금 (만원):', style=style, layout=layout_input)

# 결과 출력 위젯 생성
결과_위젯_전세 = widgets.Output(layout=layout_output)
결과_위젯_월세 = widgets.Output(layout=layout_output)
결과_위젯_비교 = widgets.Output(layout=layout_output)

# 버튼 생성
둘다_계산_버튼 = widgets.Button(description="둘 다 계산", layout=layout_button)

# 계산 함수
def combined_calculation(b):
    전세금 = 전세금_위젯.value
    연이자율 = 연이자율_위젯.value / 100
    월세보증금 = 월세_보증금_위젯.value
    월세금 = 월세금_위젯.value

    전세이자 = round((전세금 * 연이자율) / 12, 1)
    월세월이자 = round((월세보증금 * 연이자율) / 12, 1)
    실월납 = round((월세금 + 월세월이자) * 0.85, 1)

    결과_위젯_전세.clear_output()
    결과_위젯_월세.clear_output()
    결과_위젯_비교.clear_output()
    
    with 결과_위젯_전세:
        df_전세 = pd.DataFrame({
            '항목': ['전세금', '연 이자율', '전세이자(만원/월)'],
            '값': [전세금, 연이자율 * 100, 전세이자]
        }).set_index('항목')
        display(df_전세)
    
    with 결과_위젯_월세:
        df_월세 = pd.DataFrame({
            '항목': ['월세 보증금', '월세금', '월세 월 이자', '실월납(85%)'],
            '값': [월세보증금, 월세금, 월세월이자, 실월납]
        }).set_index('항목')
        display(df_월세)
    
    with 결과_위젯_비교:
        가성비 = "전세" if 전세이자 < 실월납 else "월세"
        highlighted_message = f"""
        <div style="background-color: #007BFF; color: white; padding: 10px; border-radius: 5px; text-align: center;">
            <strong style="color: yellow;">{가성비}</strong>가 월 <strong style="color: yellow;">{abs(전세이자 - 실월납):.1f}만원</strong> 더 이득!
        </div>
        """
        display(HTML(highlighted_message))

둘다_계산_버튼.on_click(combined_calculation)

# 위젯 배치
입력_위젯 = widgets.VBox([
    widgets.HBox([전세금_위젯, 연이자율_위젯]),
    widgets.HBox([월세_보증금_위젯, 월세금_위젯]),
    둘다_계산_버튼
], layout=widgets.Layout(align_items='center', width='700px'))

결과_위젯_수평배치 = widgets.HBox([결과_위젯_전세, 결과_위젯_비교, 결과_위젯_월세], layout=widgets.Layout(align_items='stretch'))

# 전체 화면 출력
main_box = widgets.VBox([입력_위젯, 결과_위젯_수평배치], layout=layout_main_box)
display(main_box)
